In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [7]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.var(dummy_data_tensor, correction = 1)
dummy_data_mean = torch.mean(dummy_data_tensor)

In [8]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<0.01*X.size()[1]*(self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*(X.size()[1]-1))<0.01*self.w*(X.size()[1]-1),x_mean_cons ),self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [9]:
# Prover/ data owner side
theory_output = torch.var(data_tensor, correction = 1)
data_mean = torch.mean(data_tensor)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
    def forward(self,X):
        # some expression of tolerance to error in the inference
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<0.01*X.size()[1]*(self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*(X.size()[1]-1))<0.01*self.w*(X.size()[1]-1),x_mean_cons ),self.w)


prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

theory output:  tensor(212.5777)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[0,1272],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1512,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [10]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.3916938304901123 seconds
Theory output:  tensor(212.5777)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 213.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [7791745561945701274, 3148105443081642255, 5687306643140073025, 2796439279989524607]]], 'proof': '02efadf3c934db2fdd4457fe096d6b6cb77f0ed84d365901bf95e334b79e4d210a405fb391924dd22707c62b7b9bc0ca081f532a090b92e01f6e92a15d5a5d5d0ac0e5560babd6af1b5c6d03da60d618ca08c5ed3ad3792b32d1fe81e415ff180bbb6301f9d2499c0e007b751688e7a1baf9f199fac553030bd92692bf46717c1f5de771d2d1b2ea53482619428a2ad6955cb0e06d9899dca7937c6c0a55b4750edc96db9e4c04222added700b5ce1e0d239e7c5eb181f7a4810d9e998c4844b0e7f5fb36bedd604aed879a39f6f9f475fc2de4765be145c9181331a8a11509f2a598b92fac8146653b4f46a12f4316dc4cfb079c32f0733cb36e2e3eb073031144f7cb0f3b22b742b41aedce679e71c5f9b6f1ed7a01ee19122a98b56d1e9ea0c70a878e5268ffdbfc567fe8e9464d2490eed7552d8989cc3978f78ea3091df0695bc70fd6b2294619e7bbcf6db0b6004c0e253bf38f2be3a7b283ce0f7cc

In [11]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [7791745561945701274, 3148105443081642255, 5687306643140073025, 2796439279989524607]]]
proof boolean:  1.0
proof result 1 : 213.0
verified
